# CPU Frequency Scaling

This demonstrates CPU frequency scaling capabilities of the PM firmware layer which enables the operating system to scale the CPU frequency up or down in order to save power. CPU frequency scaling is implemented in the Linux kernel, the infrastructure is called *cpufreq*.

1. [Introduction](#introduction)
2. [Implementation details](#implementation-details)
    1. [Kernel Config](#kernel-config)
    2. [SysFS Interface to CPUFreq](#sysfs-interface)
2. [CPU Frequency Scaling Demo](#cpufreq-scaling-demo)
3. [References](#xlnx-pm-wiki)

---

## Introduction <a name="introduction"></a>

The majority of modern processors are capable of operating in a number of different clock frequency and voltage configurations, often referred to as *Operating Performance Points (OPP)* or *P-states* (in ACPI terminology). The activity by which this happens is referred to as CPU performance scaling or CPU frequency scaling (because it involves adjusting the CPU clock frequency).

This notebook provides a user-friendly and interactive GUI to demonstrate this feature.

---

## Implementation details <a name="implementation-details"></a>

### Kernel Config <a name="kernel-config"></a>

The Linux kernel supports CPU performance scaling by means of the CPUFreq (CPU Frequency scaling) subsystem that consists of three layers of code: the core, scaling governors and scaling drivers.

More info about *cpefreq* framework in linux kernel can be found [here.](https://www.kernel.org/doc/Documentation/cpu-freq/)

The Linux kernel image binary included in Xilinx Petalinux Pre-built BSP for a Versal platform evaluation board comes with these config options enabled. In addition, the Device Tree Binary (DTB) used for the board must have a Operating Performance Points (OPP) table bindings defined with the CPU cores. Again, the pre-built binaries have this included.

Refer [this](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18841831) for reference. 

If you're compiling your own, then the following options must be enabled:
```
   - CONFIG_CPU_FREQ=y
   - CONFIG_CPU_FREQ_DEFAULT_GOV_USERSPACE=y
   - CONFIG_CPUFREQ_DT=y
```

This is a sample for DTS:
```bash
/ {
..
..
..
		cpu0: cpu@0 {
			compatible = "arm,cortex-a72", "arm,armv8";
			device_type = "cpu";
			enable-method = "psci";
			operating-points-v2 = <&cpu_opp_table>;
			reg = <0>;
			cpu-idle-states = <&CPU_SLEEP_0>;
		};

		cpu1: cpu@1 {
			compatible = "arm,cortex-a72", "arm,armv8";
			device_type = "cpu";
			enable-method = "psci";
			operating-points-v2 = <&cpu_opp_table>;
			reg = <1>;
			cpu-idle-states = <&CPU_SLEEP_0>;
		};
..
..
	};

	cpu_opp_table: cpu_opp_table {
		compatible = "operating-points-v2";
		opp-shared;
		opp00 {
			opp-hz = /bits/ 64 <1199999988>;
			opp-microvolt = <1000000>;
			clock-latency-ns = <500000>;
		};
		opp01 {
			opp-hz = /bits/ 64 <599999994>;
			opp-microvolt = <1000000>;
			clock-latency-ns = <500000>;
		};
		opp02 {
			opp-hz = /bits/ 64 <399999996>;
			opp-microvolt = <1000000>;
			clock-latency-ns = <500000>;
		};
		opp03 {
			opp-hz = /bits/ 64 <299999997>;
			opp-microvolt = <1000000>;
			clock-latency-ns = <500000>;
		};
	};
```

---

### SysFS Interface to CPUFreq <a name="sysfs-interface"></a>

During the initialization of the kernel, the CPUFreq core creates a sysfs directory called `cpufreq` under `/sys/devices/system/cpu/`.

In [1]:
ls -lah /sys/devices/system/cpu/

total 0
drwxr-xr-x    7 root     root           0 Mar  7 03:57 ./
drwxr-xr-x    6 root     root           0 Mar  7 03:57 ../
drwxr-xr-x    6 root     root           0 Mar  7 03:57 cpu0/
drwxr-xr-x    6 root     root           0 Mar  7 03:57 cpu1/
drwxr-xr-x    3 root     root           0 Mar  7 04:16 cpufreq/
drwxr-xr-x    2 root     root           0 Mar  7 04:16 hotplug/
-r--r--r--    1 root     root        4.0K Mar  7 04:16 isolated
-r--r--r--    1 root     root        4.0K Mar  7 04:16 kernel_max
-r--r--r--    1 root     root        4.0K Mar  7 04:16 modalias
-r--r--r--    1 root     root        4.0K Mar  7 04:16 offline
-r--r--r--    1 root     root        4.0K Mar  7 03:57 online
-r--r--r--    1 root     root        4.0K Mar  7 04:16 possible
drwxr-xr-x    2 root     root           0 Mar  7 04:16 power/
-r--r--r--    1 root     root        4.0K Mar  7 04:16 present
-rw-r--r--    1 root     root        4.0K Mar  7 04:16 uevent


The directory `cpufreq` above contains a `policyX` subdirectory (where `X` represents an integer number) for every policy object maintained by the CPUFreq core. The `policyX` directories in `/sys/devices/system/cpu/cpufreq` each contain policy-specific attributes (files) to control CPUFreq behavior for the corresponding policy objects (that is, for all of the CPUs associated with them).

We can see that we have only a single policy enabled, called `policy0`.

In [2]:
ls -lah /sys/devices/system/cpu/cpufreq/policy0/

total 0
drwxr-xr-x    2 root     root           0 Mar  7 04:22 ./
drwxr-xr-x    3 root     root           0 Mar  7 04:16 ../
-r--r--r--    1 root     root        4.0K Mar  7 04:22 affected_cpus
-r--------    1 root     root        4.0K Mar  7 04:22 cpuinfo_cur_freq
-r--r--r--    1 root     root        4.0K Mar  7 04:22 cpuinfo_max_freq
-r--r--r--    1 root     root        4.0K Mar  7 04:22 cpuinfo_min_freq
-r--r--r--    1 root     root        4.0K Mar  7 04:22 cpuinfo_transition_latency
-r--r--r--    1 root     root        4.0K Mar  7 04:22 related_cpus
-r--r--r--    1 root     root        4.0K Mar  7 04:22 scaling_available_frequencies
-r--r--r--    1 root     root        4.0K Mar  7 04:22 scaling_available_governors
-r--r--r--    1 root     root        4.0K Mar  7 04:22 scaling_cur_freq
-r--r--r--    1 root     root        4.0K Mar  7 04:22 scaling_driver
-rw-r--r--    1 root     root        4.0K Mar  7 04:22 scaling_governor
-rw-r--r--    1 root     root        4.0K Mar  7 04:22 sca

Each `policyX` directory is pointed to by cpufreq symbolic links under `/sys/devices/system/cpu/cpuY/` (where `Y` represents an integer that may be different from the one represented by `X`) for all of the CPUs associated with (or belonging to) the given policy.

For example:

In [3]:
ls -lah /sys/devices/system/cpu/cpu0/

total 0
drwxr-xr-x    6 root     root           0 Mar  7 03:57 ./
drwxr-xr-x    7 root     root           0 Mar  7 03:57 ../
-rw-r--r--    1 root     root        4.0K Mar  7 04:22 cpu_capacity
lrwxrwxrwx    1 root     root           0 Mar  7 04:22 cpufreq -> ../cpufreq/policy0/
drwxr-xr-x    2 root     root           0 Mar  7 04:22 hotplug/
lrwxrwxrwx    1 root     root           0 Mar  7 04:22 of_node -> ../../../../firmware/devicetree/base/cpus/cpu@0/
-rw-r--r--    1 root     root        4.0K Mar  7 04:22 online
drwxr-xr-x    2 root     root           0 Mar  7 04:22 power/
drwxr-xr-x    3 root     root           0 Mar  7 04:22 regs/
lrwxrwxrwx    1 root     root           0 Mar  7 04:22 subsystem -> ../../../../bus/cpu/
drwxr-xr-x    2 root     root           0 Mar  7 04:22 topology/
-rw-r--r--    1 root     root        4.0K Mar  7 03:57 uevent


There are quite a few attributes listed under under `/sys/devices/system/cpu/cpufreq/policyX/`, in our case `policy0`.
Lets look at a few important ones from demo perspective.

- **`affected_cpus`**: List of online CPUs belonging to this policy.

- **`cpuinfo_cur_freq`**: This is expected to be the frequency (in KHz) the hardware actually runs at (obtained from hardware).

- **`cpuinfo_max_freq`**: Maximum possible operating frequency the CPUs belonging to this policy can run at (in kHz).

- **`cpuinfo_min_freq`**: Minimum possible operating frequency the CPUs belonging to this policy can run at (in kHz).

- **`cpuinfo_transition_latency`**: he time it takes to switch the CPUs belonging to this policy from one P-state to another, in nanoseconds.

- **`related_cpus`**: List of all (online and offline) CPUs belonging to this policy.

- **`scaling_available_governors`**: List of CPUFreq scaling governors present in the kernel that can be attached to this policy.

- **`scaling_cur_freq`**: Current frequency of all of the CPUs belonging to this policy (in kHz).
    In the majority of cases, this is the frequency of the last P-state requested by the scaling driver from the hardware using the scaling interface provided by it, which may or may not reflect the frequency the CPU is actually running at (due to hardware design and other limitations).

- **`scaling_driver`**: The scaling driver currently in use.

- **`scaling_governor`**: The scaling governor currently attached to this policy.

- **`scaling_max_freq`**: Maximum frequency the CPUs belonging to this policy are allowed to be running at (in kHz).
    This attribute is read-write and writing a string representing an integer to it will cause a new limit to be set (it must not be lower than the value of the scaling_min_freq attribute).

- **`scaling_min_freq`**: Minimum frequency the CPUs belonging to this policy are allowed to be running at (in kHz).
    This attribute is read-write and writing a string representing a non-negative integer to it will cause a new limit to be set (it must not be higher than the value of the scaling_max_freq attribute).

- **`scaling_setspeed`**: This attribute is functional only if the userspace scaling governor is attached to the given policy.
    It returns the last frequency requested by the governor (in kHz) or can be written to in order to set a new frequency for the policy.

More information about `cpufreq` sysfs interface can be found [here.](https://www.kernel.org/doc/Documentation/cpu-freq/user-guide.txt)

The CPU cores of the system are currently running at this frequency:

In [4]:
cat /sys/devices/system/cpu/cpufreq/policy0/cpuinfo_cur_freq | xargs printf "%s KHz"

999999 KHz

The minimum and maximum CPU frequency range supported by *hardware* is:

In [5]:
!cat /sys/devices/system/cpu/cpufreq/policy0/cpuinfo_min_freq | xargs printf "%s KHz - ";
!cat /sys/devices/system/cpu/cpufreq/policy0/cpuinfo_max_freq | xargs printf "%s KHz"

299999 KHz - 1199999 KHz

**<font color=blue> For our demo, following is required: </font>**
- `scaling_governor` in use must be *userspace*
- `scaling_min_freq` must be greater than or equal to `cpuinfo_min_freq`
- `scaling_max_freq` must be less than or equal to `cpuinfo_max_freq`

Let's verify that.

In [6]:
cat /sys/devices/system/cpu/cpufreq/policy0/scaling_governor

userspace


In [7]:
!cat /sys/devices/system/cpu/cpufreq/policy0/scaling_min_freq | xargs printf "%s KHz - ";
!cat /sys/devices/system/cpu/cpufreq/policy0/scaling_max_freq | xargs printf "%s KHz"

299999 KHz - 1199999 KHz

---

## CPU Frequency Scaling Demo  <a name="cpufreq-scaling-demo"></a>

The demo will allow us to interactively change the scaling frequency by writing to `scaling_setspeed`.

Let us see what are the available scaling frequencies on the system:

In [8]:
cat /sys/devices/system/cpu/cpufreq/policy0/scaling_available_frequencies

299999 399999 599999 1199999 


Let us try the demo:

In [ ]:
from pmutil import cpufreq
cpufreq.run_demo()

Run the following after every frequency change to verify the difference between execution times:

In [ ]:
%timeit -n 10000 -r 10 import random; random.randint(0,1000)

## References <a name="xlnx-pm-wiki"></a>
[Link to Xilinx PM Wiki Page Zynq UltraScale＋](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842232)

[Link to Xilinx PM Frequency Scaling Zynq UltraScale＋](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18841831)